In [ ]:
# numpy downgrade가 필요할 수 있음
#!python -m pip install numpy==1.2

In [1]:
#!pip install ensemble_boxes

     |████████████████████████████████| 3.3 MB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 26.3 MB 88.5 MB/s eta 0:00:01


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [16]:
# ensemble csv files
submission_files = ['/opt/ml/detection/mmdetection_new/work_dirs/swinB_512_1024_zero_thres.csv',
                    '/opt/ml/detection/mmdetection_new/work_dirs/swinB_smalldb_fold1/swinB_smalldb_fold1_submission_.csv'
                    ] # submission lists
# submission_files = ['/opt/ml/detection/mmdetection_new/work_dirs/swinB_512_1024_zero_thres.csv',
#                     '/opt/ml/detection/mmdetection_new/boostcamp/work_dirs/multi_scale_HTC_swinB_pafpn/1/multi_scale_HTC_swinB_pafpn_submission_.csv'
#                     ] # submission lists

submission_df = [pd.read_csv(file) for file in submission_files]

image_ids = submission_df[0]['image_id'].tolist()

annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [19]:
prediction_strings = []
file_names = []
iou_thr = 0.4

ensemble_function = weighted_boxes_fusion

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = ensemble_function(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [20]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('swinB_cascade_swinB_small_cascade_wbf.csv')

submission.head()

,PredictionString,image_id
0,5 0.642518937587738 244.25326538085938 682.018...,test/0000.jpg
1,5 0.49741634726524353 130.90673828125 1.561286...,test/0001.jpg
2,1 0.5781958699226379 676.7100830078125 229.842...,test/0002.jpg
3,9 0.4998994767665863 153.88446044921875 265.30...,test/0003.jpg
4,0 0.31084781885147095 423.2960205078125 407.36...,test/0004.jpg


## nms + soft_nms + wbf + nmw

In [12]:
# ensemble csv files
submission_files = ['swinB_cascade_swinB_HTC_nmw.csv',
                    'swinB_cascade_swinB_HTC_nms.csv',
                    'swinB_cascade_swinB_HTC_soft_nms.csv',
                    'swinB_cascade_swinB_HTC_wbf.csv'
                    ] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

image_ids = submission_df[0]['image_id'].tolist()

annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
prediction_strings = []
file_names = []
iou_thr = 0.4

ensemble_function =  soft_nms

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = ensemble_function(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Che

Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


In [14]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('swinB_cascade_swinB_HTC_total_soft_nms.csv')

submission.head()

,PredictionString,image_id
0,0 0.30352563 489.4098815917969 585.01861572265...,test/0000.jpg
1,0 0.17676884 759.9013671875 663.2786865234375 ...,test/0001.jpg
2,0 0.9725914 880.3657836914062 462.399993896484...,test/0002.jpg
3,0 0.048839312 6.2980804443359375 0.34271004796...,test/0003.jpg
4,0 0.93200076 424.5522766113281 406.36660766601...,test/0004.jpg
